In [600]:
#import dask
import xarray as xr
import concurrent.futures

In [601]:
from dask.distributed import Client
client = Client(processes=False)
client

ModuleNotFoundError: No module named 'dask'

In [8]:
# set test nc path, could be any
file = r"C:\Users\Lewis\Downloads\test.nc"

In [9]:
# try load ds without dask, time it
%time ds_raw = xr.open_rasterio(file)

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
<timed exec> in <module>

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\xarray\backends\rasterio_.py in open_rasterio(filename, parse_coordinates, chunks, cache, lock)
    211         The newly created DataArray.
    212     """
--> 213     import rasterio
    214     from rasterio.vrt import WarpedVRT
    215 

ModuleNotFoundError: No module named 'rasterio'


In [10]:
# try load ds with dask, time it
chunks = {'band': 1, 'x': 'auto', 'y': 'auto'}
%time ds_dask = xr.open_rasterio(file, chunks=chunks)

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
<timed exec> in <module>

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\xarray\backends\rasterio_.py in open_rasterio(filename, parse_coordinates, chunks, cache, lock)
    211         The newly created DataArray.
    212     """
--> 213     import rasterio
    214     from rasterio.vrt import WarpedVRT
    215 

ModuleNotFoundError: No module named 'rasterio'


In [11]:
# try compute regular
%time ds_test = ds_dask.compute(scheduler='processes')

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<timed exec> in <module>

NameError: name 'ds_dask' is not defined


In [12]:
# try compute regular
%time ds_test = ds_dask.compute(processes=False)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<timed exec> in <module>

NameError: name 'ds_dask' is not defined


In [16]:
# this method does not need distributed and is quick
# also, concurrent is built in to arcgispro!
# needs a dask ds to work
import concurrent.futures 

# try load ds with dask, time it
chunks = {'band': 1, 'x': 'auto', 'y': 'auto'}
ds_dask = xr.open_rasterio(file, chunks=chunks)

# create compute func
def compute_da(da):
    return da.compute()


# split ds into seperate das
da_list = []
for dt in ds_dask['band']:
    da = ds_dask.sel(band=dt)
    da_list.append(da)
    
# try parallel load of all bands
num_cores = 8
with concurrent.futures.ThreadPoolExecutor(num_cores) as executor:
    da_list = list(executor.map(compute_da, da_list))
    
ds = xr.concat(da_list, dim='band')

In [17]:
ds

<xarray.DataArray (band: 109, y: 900, x: 500)>
array([[[ 943,  940,  973, ..., 1005,  998,  995],
        [ 880,  974, 1043, ..., 1003,  991,  978],
        [ 912,  947, 1083, ..., 1034,  975,  979],
        ...,
        [1002,  959,  985, ...,  964,  923,  975],
        [1033,  988, 1043, ...,  941,  998, 1083],
        [ 968, 1021, 1013, ..., 1048, 1069, 1011]],

       [[3540, 3560, 3598, ..., 4697, 4651, 4627],
        [3596, 3633, 3597, ..., 4800, 4764, 4693],
        [3600, 3647, 3657, ..., 4910, 4785, 4646],
        ...,
        [5017, 4766, 4371, ..., 5303, 5475, 5545],
        [4990, 4798, 4577, ..., 5397, 5497, 5660],
        [5015, 4810, 4707, ..., 5480, 5625, 5717]],

       [[ 959,  888,  957, ...,  907,  963,  956],
        [ 957,  958, 1068, ...,  953,  970,  988],
        [ 890,  893, 1144, ..., 1002,  975,  947],
        ...,
...
        ...,
        [1227, 1096, 1091, ...,  974,  935,  983],
        [1136, 1094, 1116, ...,  891, 1035, 1145],
        [1133, 1153, 1145, ..., 1050, 1162, 1139]],

       [[2792, 3100, 3111, ..., 2359, 2412, 2412],
        [3019, 3110, 3080, ..., 2261, 2179, 2056],
        [3022, 3038, 2929, ..., 1939, 1996, 2003],
        ...,
        [1172, 1158, 1093, ..., 1029, 1055, 1076],
        [1198, 1154, 1176, ..., 1007, 1094, 1175],
        [1164, 1214, 1146, ..., 1077, 1160, 1168]],

       [[ 891,  829,  892, ...,  852,  915,  948],
        [ 800,  832,  991, ...,  858,  886,  875],
        [ 831,  896, 1091, ...,  968,  909,  880],
        ...,
        [ 857,  815,  750, ...,  924,  822,  871],
        [ 854,  843,  898, ...,  869,  892, 1005],
        [ 790,  873,  897, ...,  908, 1022, 1061]]], dtype=int16)
Coordinates:
  * band     (band) int64 1 2 3 4 5 6 7 8 9 ... 102 103 104 105 106 107 108 109
  * y        (y) float64 -2.538e+06 -2.538e+06 ... -2.565e+06 -2.565e+06
  * x        (x) float64 6.774e+05 6.774e+05 6.775e+05 ... 6.923e+05 6.924e+05
Attributes:
    transform:   (30.0, 0.0, 677385.0, 0.0, -30.0, -2537685.0)
    res:         (30.0, 30.0)
    is_tiled:    0
    nodatavals:  (-32767.0, -32767.0, -32767.0, -32767.0, -32767.0, -32767.0,...
    scales:      (1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...
    offsets:     (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...